In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from scipy.stats import levene
from scipy.stats import kstest
from statsmodels.stats.anova import anova_lm#方差分析
from statsmodels.stats.multicomp import pairwise_tukeyhsd#两两比较
import pingouin as pg
import re
from scipy.optimize import curve_fit
import palettable
from palettable.cartocolors.qualitative import Bold_9

# matplotlib 图形正常显示中文及负号
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号
parameters = {'axes.labelsize': 15,
          'axes.titlesize': 20,'xtick.labelsize':13,'ytick.labelsize':13}
plt.rcParams.update(parameters)
#3d绘图内嵌可旋转
#%matplotlib notebook
#3d绘图单独窗口
%matplotlib qt5
%config InlineBackend.figure_format = 'svg'#矢量图设置，设定显示图片的分辨率

pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

In [2]:
def review_count(row):
    return [float(x) for x in row.strip('[]').split()]

def review_compute1(row):
    if row['评论1_lag1']:
        return list(np.array(row['评论1'])-np.array(row['评论1_lag1']))
    else:
        return row['评论1']
               
def review_compute2(row):
    if row['评论2_lag1']:
        return list(np.array(row['评论2'])-np.array(row['评论2_lag1']))
    else:
        return row['评论2']

def create_lag_column(group):
    group['评论1_lag1'] = group['评论1'].shift(1)
    group['评论2_lag1'] = group['评论2'].shift(1)
    group.loc[1:,['评论1_当期']]=group.loc[1:,['评论1','评论1_lag1']].apply(review_compute1,axis=1)
    group.loc[1:,['评论2_当期']]=group.loc[1:,['评论2','评论2_lag1']].apply(review_compute2,axis=1)
    return group

def review_perception1(row):
    b_weight=10
    b=b_weight*np.array([0.11,-0.15,0.17,0,0,-0.23])
    if row['评论1_lag1']:
        return np.dot(np.array(row['评论1_lag1'])/(sum(np.array(row['评论1_lag1']))), b.T)
    else:
        return 0
    
def review_perception2(row):
    b_weight=10
    b=b_weight*np.array([0.11,-0.15,0.17,0,0,-0.23])
    if row['评论2_lag1']:
        return np.dot(np.array(row['评论2_lag1'])/(sum(np.array(row['评论2_lag1']))), b.T)
    else:
        return 0

# 仿真结果

In [3]:
data=pd.read_csv(r'C:\Users\付应超\Desktop\小论文数据-在线评论\数据-jais修改\数据\merge_data_modify.csv')
data['平均好评激励金额']=(data['好评激励1']+data['好评激励2'])/2
data['平均夸大宣传程度']=(data['夸大宣传程度1']+data['夸大宣传程度2'])/2
data['平均回应偏好']=(data['回应偏好1']+data['回应偏好2'])/2
data['平均价格']=(data['价格1']+data['价格2'])/2
data['平均服务质量']=(data['服务质量1']+data['服务质量2'])/2
data['平均毛利率']=data['平均价格']-data['平均服务质量']
data['平均净利率']=(1-0.15)*data['平均价格']-data['平均服务质量']
data['毛利率1']=data['价格1']-data['服务质量1']
data['毛利率2']=data['价格2']-data['服务质量2']
data['净利率1']=(1-0.15)*data['价格1']-data['服务质量1']
data['净利率2']=(1-0.15)*data['价格2']-data['服务质量2']
data['消费者平均收益1']=data['消费者收益1']/data['服务人数1']
data['消费者平均收益2']=data['消费者收益2']/data['服务人数2']
data['评论1']=data['评论1'].apply(review_count)
data['评论2']=data['评论2'].apply(review_count)
data['评论1_当期']=data['评论1']
data['评论2_当期']=data['评论2']
data=data.groupby(['提供方1类型', '提供方2类型','回应偏好1', '回应偏好2']).apply(create_lag_column)
data.reset_index(drop=True, inplace=True)
data['总评论数量1']=data['评论1_当期'].apply(sum)
data['总评论数量2']=data['评论2_当期'].apply(sum)
data['总评论数量']=data['总评论数量1']+data['总评论数量2']
data['好评数量1']=data['评论1_当期'].apply(lambda x:x[0]+x[2]+x[4])
data['好评数量2']=data['评论2_当期'].apply(lambda x:x[0]+x[2]+x[4])
data['好评总数']=data['好评数量1']+data['好评数量2']
data['当期好评占比1']=data['好评数量1']/data['总评论数量1']
data['当期好评占比2']=data['好评数量2']/data['总评论数量2']
data['被激励好评占好评比例1']=data['被激励好评数量1']/data['好评数量1']
data['被激励好评占好评比例2']=data['被激励好评数量2']/data['好评数量2']
data['被激励好评占好评比例']=(data['被激励好评占好评比例1']+data['被激励好评占好评比例2'])/2
data['获得激励评论数量占评论比例1']=data['获得激励数量1']/data['总评论数量1']
data['获得激励评论数量占评论比例2']=data['获得激励数量2']/data['总评论数量2']
data['获得激励评论数量占评论比例']=(data['获得激励评论数量占评论比例1']+data['获得激励评论数量占评论比例2'])/2
data['评论感知1']=data.apply(review_perception1,axis=1)
data['评论感知2']=data.apply(review_perception2,axis=1)
data['val1']= (0.3 + data['服务质量1'] - data['价格1'])
data['expect1']=(0.35+0.65*data['评论感知1'])*(0.3 + (1 + data['夸大宣传程度1']) * (data['服务质量1']) - data['价格1'])
data['val2']= (0.3 + data['服务质量2'] - data['价格2'])
data['expect2']=(0.35+0.65*data['评论感知2'])*(0.3 + (1 + data['夸大宣传程度2']) * (data['服务质量2']) - data['价格2'])
data=data[data['step']>1]
data.columns

C:\Users\付应超\AppData\Local\Temp\ipykernel_4480\524130988.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data=data.groupby(['提供方1类型', '提供方2类型','回应偏好1', '回应偏好2']).apply(create_lag_column)


Index(['step', '提供方1类型', '提供方2类型', '提供方1利他程度', '提供方2利他程度', '夸大宣传程度1', '夸大宣传程度2', '好评激励1', '好评激励2', '价格1', '价格2', '服务质量1', '服务质量2', '服务人数1', '服务人数2', '总服务人数', '评论1', '评论2', '社会总福利', '平台收益', '提供方收益1', '提供方收益2', '提供方总收益', '消费者收益1', '消费者收益2', '消费者总收益', '实际到达人数', '办理入住数1', '办理入住数2', '办理入住总数', '被激励好评数量1', '被激励好评数量2', '获得激励数量1', '获得激励数量2', '好评占比1', '好评占比2', '总好评数量占比', '回应偏好1', '回应偏好2', 'market_type', '平均好评激励金额', '平均夸大宣传程度', '平均回应偏好', '平均价格', '平均服务质量', '平均毛利率', '平均净利率', '毛利率1', '毛利率2', '净利率1', '净利率2', '消费者平均收益1', '消费者平均收益2', '评论1_当期', '评论2_当期', '评论1_lag1', '评论2_lag1', '总评论数量1', '总评论数量2', '总评论数量', '好评数量1', '好评数量2', '好评总数', '当期好评占比1', '当期好评占比2', '被激励好评占好评比例1', '被激励好评占好评比例2', '被激励好评占好评比例', '获得激励评论数量占评论比例1', '获得激励评论数量占评论比例2', '获得激励评论数量占评论比例', '评论感知1', '评论感知2', 'val1', 'expect1', 'val2', 'expect2'], dtype='object')

In [4]:
data.loc[(data['提供方1类型']==0) &(data['提供方2类型']==0),['market_type']]='pure-egoism'
data.loc[(data['提供方1类型']==0) &(data['提供方2类型']==1),['market_type']]='hybrid'
data.loc[(data['提供方1类型']==1) &(data['提供方2类型']==1),['market_type']]='pure-altruism'

In [5]:
data=data.rename(columns={'夸大宣传程度1':'OP1','夸大宣传程度2':'OP2','好评激励1':'PCIP1','好评激励2':'PCIP2',
                '回应偏好1':'PERP1','回应偏好2':'PERP2','价格1':'ARPP1','价格2':'ARPP2','服务质量1':'ARSP1','服务质量2':'ARSP2',
                '平均夸大宣传程度':'OP','平均好评激励金额':'PCIP','平均回应偏好':'PERP','平均价格':'ARPP','平均服务质量':'ARSP',
                '评论感知1':'RP1','评论感知2':'RP2'})

In [6]:
def reveiw_percent(row):
    row['累计评论数量1']=sum(row['评论1'])
    row['累计评论数量2']=sum(row['评论2'])
    row['好评无回应占比1']=row['评论1'][0]/row['累计评论数量1']
    row['差评无回应占比1']=row['评论1'][1]/row['累计评论数量1']
    row['价值共创占比1']=row['评论1'][2]/row['累计评论数量1']
    row['评论端价值下降占比1']=row['评论1'][3]/row['累计评论数量1']
    row['回应端价值下降占比1']=row['评论1'][4]/row['累计评论数量1']
    row['价值共毁占比1']=row['评论1'][5]/row['累计评论数量1']
    row['好评无回应占比2']=row['评论2'][0]/row['累计评论数量2']
    row['差评无回应占比2']=row['评论2'][1]/row['累计评论数量2']
    row['价值共创占比2']=row['评论2'][2]/row['累计评论数量2']
    row['评论端价值下降占比2']=row['评论2'][3]/row['累计评论数量2']
    row['回应端价值下降占比2']=row['评论2'][4]/row['累计评论数量2']
    row['价值共毁占比2']=row['评论2'][5]/row['累计评论数量2']
    row['对好评回应率1']=(row['评论1'][2]+row['评论1'][4])/(row['评论1'][0]+row['评论1'][2]+row['评论1'][4])
    row['对差评回应率1']=(row['评论1'][3]+row['评论1'][5])/(row['评论1'][1]+row['评论1'][3]+row['评论1'][5])
    row['对好评提升型回应率1']=row['评论1'][2]/(row['评论1'][2]+row['评论1'][4])
    row['对差评提升型回应率1']=row['评论1'][3]/(row['评论1'][3]+row['评论1'][5])
    row['累计好评占比1']=(row['评论1'][0]+row['评论1'][2]+row['评论1'][4])/row['累计评论数量1']
    return row
data=data[(data['market_type']=='pure-egoism') & (data['PERP1']==0.5) & (data['PERP2']==0.5)]
data=data.apply(reveiw_percent,axis=1)

In [7]:
data.describe()

,step,提供方1类型,提供方2类型,提供方1利他程度,提供方2利他程度,OP1,OP2,PCIP1,PCIP2,ARPP1,ARPP2,ARSP1,ARSP2,服务人数1,服务人数2,总服务人数,社会总福利,平台收益,提供方收益1,提供方收益2,提供方总收益,消费者收益1,消费者收益2,消费者总收益,实际到达人数,办理入住数1,办理入住数2,办理入住总数,被激励好评数量1,被激励好评数量2,获得激励数量1,获得激励数量2,好评占比1,好评占比2,总好评数量占比,PERP1,PERP2,PCIP,OP,PERP,ARPP,ARSP,平均毛利率,平均净利率,毛利率1,毛利率2,净利率1,净利率2,消费者平均收益1,消费者平均收益2,总评论数量1,总评论数量2,总评论数量,好评数量1,好评数量2,好评总数,当期好评占比1,当期好评占比2,被激励好评占好评比例1,被激励好评占好评比例2,被激励好评占好评比例,获得激励评论数量占评论比例1,获得激励评论数量占评论比例2,获得激励评论数量占评论比例,RP1,RP2,val1,expect1,val2,expect2,累计评论数量1,累计评论数量2,好评无回应占比1,差评无回应占比1,价值共创占比1,评论端价值下降占比1,回应端价值下降占比1,价值共毁占比1,好评无回应占比2,差评无回应占比2,价值共创占比2,评论端价值下降占比2,回应端价值下降占比2,价值共毁占比2,对好评回应率1,对差评回应率1,对好评提升型回应率1,对差评提升型回应率1,累计好评占比1
count,198.00000,198.0,198.0,198.0,198.0,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.0,198.0,198.000000,198.000000,198.0,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000
mean,51.00000,0.0,0.0,0.0,0.0,0.153510,0.153510,0.079916,0.079916,0.568730,0.568730,0.246607,0.246607,121.004914,121.004914,242.009828,254.278426,63.776107,72.824715,72.824715,145.649430,22.426445,22.426445,44.852889,281.267657,120.990247,120.990247,241.980495,61.393970,61.393970,80.067429,80.067429,0.820390,0.820390,0.819528,0.5,0.5,0.079916,0.153510,0.5,0.568730,0.246607,0.322123,0.236814,0.322123,0.322123,0.236814,0.236814,0.185806,0.185806,98.024616,98.024616,196.049232,80.067429,80.067429,160.134859,0.819821,0.819821,0.756470,0.756470,0.756470,0.819821,0.819821,0.819821,0.738890,0.738890,-0.022123,0.012797,-0.022123,0.012797,4990.947591,4990.947591,0.657319,0.112195,0.152456,0.034068,0.009860,0.034102,0.657319,0.112195,0.152456,0.034068,0.009860,0.034102,0.198034,0.377884,0.939258,0.499690,0.819634
std,28.64982,0.0,0.0,0.0,0.0,0.044272,0.044272,0.025136,0.025136,0.109374,0.109374,0.101429,0.101429,7.106736,7.106736,6.158345,6.767688,8.830153,8.153248,8.153248,9.871847,6.350368,6.350368,9.655781,0.812902,8.147983,8.147983,6.783515,19.745339,19.745339,13.267594,13.267594,0.024864,0.024864,0.020736,0.0,0.0,0.017080,0.031890,0.0,0.072977,0.071106,0.013326,0.015177,0.021888,0.021888,0.020620,0.020620,0.052817,0.052817,11.038079,11.038079,15.812260,13.267594,13.267594,19.339440,0.115344,0.115344,0.166088,0.166088,0.119919,0.115344,0.115344,0.088433,0.070069,0.070069,0.021888,0.017766,0.021888,0.017766,2817.108295,2817.108295,0.019306,0.014958,0.004494,0.004585,0.000309,0.004555,0.019306,0.014958,0.004494,0.004585,0.000309,0.004555,0.000262,0.001153,0.000361,0.000880,0.024096
min,2.00000,0.0,0.0,0.0,0.0,0.051155,0.051155,0.020687,0.020687,0.335096,0.335096,0.036937,0.036937,102.196000,102.196000,229.991000,237.310142,46.020960,56.001098,56.001098,124.266783,8.439699,8.439699,27.938355,278.572000,99.011000,99.011000,228.773000,12.758000,12.758000,41.385000,41.385000,0.782574,0.782574,0.801876,0.5,0.5,0.041166,0.073047,0.5,0.386208,0.073369,0.271388,0.194806,0.263574,0.263574,0.172255,0.172255,0.063132,0.063132,72.330000,72.330000,165.570000,41.385000,41.385000,115.326000,0.391022,0.391022,0.190985,0.190985,0.419832,0.391022,0.391022,0.547487,0.636857,0.636857,-0.102230,-0.071345,-0.102230,-0.071345,168.090000,168.090000,0.626555,0.036409,0.145369,0.010607,0.009314,0.010596,0.626555,0.036409,0.145369,0.010607,0.009314,0.010596,0.197521,0.368030,0.937388

In [8]:
plt.rcParams['font.sans-serif'] = ['SimHei']
fig, ax = plt.subplots(figsize=(10, 6))
dt=data
#gb=dt.groupby(['PERP1','PERP2'])
#dt=gb.get_group((0.9, 0.9))
x='好评无回应占比1'
y='办理入住数1'
sns.scatterplot(x=x, y=y, data=dt, ax=ax)
sns.regplot(x=x, y=y, data=dt, order=5, scatter=False, color='red', ax=ax)
# 手动设置y轴的显示范围
#ax.set_ylim(0.1, 0.2)  # 根据你的数据和需要调整这些值
plt.xlabel(x)
plt.ylabel(y)
plt.title('Scatter Plot of Step vs Value with Non-linear Fit')
plt.show()

In [281]:
plt.rcParams['font.sans-serif'] = ['SimHei']
fig, ax = plt.subplots(figsize=(10, 6))
dt=data
#gb=dt.groupby(['PERP1','PERP2'])
#dt=gb.get_group((0.9, 0.9))
x='step'
y='ARPP1'
sns.scatterplot(x=x, y=y, data=dt, ax=ax)
sns.regplot(x=x, y=y, data=dt, order=5, scatter=False, color='red', ax=ax)
x='step'
y='ARPP2'
sns.scatterplot(x=x, y=y, data=dt, ax=ax)
sns.regplot(x=x, y=y, data=dt, order=5, scatter=False, color='red', ax=ax)
# 手动设置y轴的显示范围
ax.set_ylim(-1, 2)  # 根据你的数据和需要调整这些值
plt.xlabel(x)
plt.ylabel('ARPP')
plt.title('Scatter Plot of Step vs Value with Non-linear Fit')
plt.show()

# 现实airbnb评论数据

In [4]:
data2=pd.read_csv(r'data_merge_all2.csv')
data2=data2[['room_id','response_num','comment_num','positive_c_num','negtive_c_num','positive_r_num',
           'negtive_r_num','co_create_num','c_decline_num','r_decline_num','co_destruction_num']]
data2['positive_c_without_r']=data2['positive_c_num']-data2['co_create_num']-data2['r_decline_num']
data2['negtive_c_without_r']=data2['negtive_c_num']-data2['co_destruction_num']-data2['c_decline_num']
gb=data2.groupby('room_id').sum().reset_index()
#gb=data2.groupby('room_id').filter(lambda x: len(x) > 10).groupby('room_id').sum().reset_index()
#gb=gb[gb['response_num']>0]
gb['reviews'] = gb.apply(lambda row: [row['positive_c_without_r'], row['negtive_c_without_r'], row['co_create_num'], row['c_decline_num'], row['r_decline_num'], row['co_destruction_num']], axis=1)
gb['好评无回应占比'] = gb['reviews'].apply(lambda x: x[0]/sum(x))
gb['差评无回应占比'] = gb['reviews'].apply(lambda x: x[1]/sum(x))
gb['价值共创占比'] = gb['reviews'].apply(lambda x: x[2]/sum(x))
gb['评论端价值下降占比'] = gb['reviews'].apply(lambda x: x[3]/sum(x))
gb['回应端价值下降占比'] = gb['reviews'].apply(lambda x: x[4]/sum(x))
gb['价值共毁占比'] = gb['reviews'].apply(lambda x: x[5]/sum(x))

#对于每条好评，商家回应的概率
gb['r_to_pc']=(gb['co_create_num']+gb['r_decline_num'])/gb['positive_c_num']
#对于每条差评，商家回应的概率
gb['r_to_nc']=(gb['co_destruction_num']+gb['c_decline_num'])/gb['negtive_c_num']
#对于每条好评，商家做出好的回应的概率
gb['pr_to_pc']=gb['co_create_num']/(gb['co_create_num']+gb['r_decline_num'])
#对于每条差评，商家做出好的回应的概率
gb['pr_to_nc']=gb['c_decline_num']/(gb['co_destruction_num']+gb['c_decline_num'])
#好评占比
gb['pr_P']=gb['positive_c_num']/gb['comment_num']


gb.describe()

,room_id,response_num,comment_num,positive_c_num,negtive_c_num,positive_r_num,negtive_r_num,co_create_num,c_decline_num,r_decline_num,co_destruction_num,positive_c_without_r,negtive_c_without_r,好评无回应占比,差评无回应占比,价值共创占比,评论端价值下降占比,回应端价值下降占比,价值共毁占比,r_to_pc,r_to_nc,pr_to_pc,pr_to_nc,pr_P
count,2.596800e+04,25968.000000,25968.000000,25968.000000,25968.000000,25968.000000,25968.000000,25968.000000,25968.000000,25968.000000,25968.000000,25968.000000,25968.000000,25962.000000,25962.000000,25962.000000,25962.000000,25962.000000,25962.000000,25900.000000,8692.000000,12958.000000,4244.00000,25968.000000
mean,3.166022e+07,4.443315,19.933495,19.225200,0.644678,4.133395,0.286198,4.003273,0.118030,0.160005,0.123614,15.061922,0.403035,0.772000,0.025060,0.185677,0.005876,0.005936,0.005452,0.198119,0.377891,0.938697,0.48647,0.960394
std,6.854245e+06,10.904912,26.540186,25.747517,1.316195,10.449515,1.001346,10.276455,0.421312,0.809078,0.436220,22.196206,0.976282,0.301107,0.079548,0.290152,0.034755,0.033219,0.030012,0.302067,0.433746,0.192008,0.46365,0.095117
min,3.507280e+05,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,2.746917e+07,0.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.958333
50%,3.329488e+07,1.000000,10.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.911765,0.000000,0.000000,0.000000,0.000000,0.000000,0.006494,0.000000,1.000000,0.50000,1.000000
75%,3.713189e+07,4.000000,25.000000,24.000000,1.000000,3.000000,0.000000,3.000000,0.000000,0.000000,0.000000,18.000000,0.000000,1.000000,0.000000,0.285714,0.000000,0.000000,0.000000,0.307692,1.000000,1.000000,1.00000,1.000000
max,4.116819e+07,179.000000,258.000000,254.000000,40.000000,171.000000,32.000000,168.000000,9.000000,31.000000,7.000000,254.000000,32.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [19]:
sample = np.array([0.198034, 0.377884, 0.939258, 0.499690])
#sample = np.array([0.819634])
# 计算欧几里得距离
#distances = np.linalg.norm(gb[['pr_P']].values - sample, axis=1)
distances = np.linalg.norm(gb[['r_to_pc', 'r_to_nc', 'pr_to_pc', 'pr_to_nc']].values - sample, axis=1)
# 添加距离到DataFrame
gb['distance'] = distances
# 排序并选择前10个最相似的样本
top_10_matches = gb.sort_values(by='distance').head(5)
top_10_matches.describe()

,room_id,response_num,comment_num,positive_c_num,negtive_c_num,positive_r_num,negtive_r_num,co_create_num,c_decline_num,r_decline_num,co_destruction_num,positive_c_without_r,negtive_c_without_r,好评无回应占比,差评无回应占比,价值共创占比,评论端价值下降占比,回应端价值下降占比,价值共毁占比,r_to_pc,r_to_nc,pr_to_pc,pr_to_nc,pr_P,distance
count,5.000000e+00,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.0,5.000000,5.0,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.0,5.000000,5.000000
mean,2.556145e+07,14.200000,69.400000,64.000000,5.400000,13.000000,1.200000,12.000000,1.0,0.200000,1.0,51.800000,3.400000,0.716206,0.067597,0.173040,0.020803,0.001550,0.020803,0.196299,0.373333,0.990000,0.5,0.890796,0.071544
std,4.720689e+06,6.870226,39.525941,39.242834,0.547723,6.595453,0.447214,6.595453,0.0,0.447214,0.0,33.056013,0.547723,0.077883,0.048649,0.038431,0.016753,0.003467,0.016753,0.037781,0.036515,0.022361,0.0,0.081906,0.010524
min,1.857084e+07,5.000000,20.000000,15.000000,5.000000,4.000000,1.000000,3.000000,1.0,0.000000,1.0,12.000000,3.000000,0.600000,0.031008,0.147287,0.007752,0.000000,0.007752,0.161290,0.333333,0.950000,0.5,0.750000,0.057928
25%,2.321094e+07,12.000000,55.000000,49.000000,5.000000,11.000000,1.000000,10.000000,1.0,0.000000,1.0,39.000000,3.000000,0.697368,0.039474,0.149254,0.013158,0.000000,0.013158,0.162602,0.333333,1.000000,0.5,0.890909,0.064674
50%,2.681856e+07,12.000000,67.000000,62.000000,5.000000,11.000000,1.000000,10.000000,1.0,0.000000,1.0,52.000000,3.000000,0.709091,0.044776,0.150000,0.014925,0.000000,0.014925,0.200000,0.400000,1.000000,0.5,0.925373,0.074357
75%,2.910851e+07,20.000000,76.000000,71.000000,6.000000,19.000000,1.000000,18.000000,1.0,0.000000,1.0,53.000000,4.000000,0.776119,0.072727,0.181818,0.018182,0.000000,0.018182,0.204082,0.400000,1.000000,0.5,0.934211,0.075571
max,3.009842e+07,22.000000,129.000000,123.000000,6.000000,20.000000,2.000000,19.000000,1.0,1.000000,1.0,103.000000,4.000000,0.798450,0.150000,0.236842,0.050000,0.007752,0.050000,0.253521,0.400000,1.000000,0.5,0.953488,0.085192


In [16]:
0.819634*0.198034

0.162315399556

# 现实airbnb定价数据

###### 方法一：针对于现实数据中价格呈现的正拖尾现象，本研究对价格取对数后归一化

In [185]:
data_r=pd.read_csv(r'data_merge_all2.csv')
data_r=data_r[['room_id','price']]
data_r=data_r.groupby('room_id').mean().reset_index()
data_r = pd.merge(data_r, gb, on='room_id', how='inner')
data_r['price'] = np.log1p(data_r['price']) #对数转换减少正拖尾的影响

In [186]:
sample = np.array([0.198034, 0.377884, 0.939258, 0.499690,0.819634])
#sample = np.array([0.657319, 0.112195, 0.152456, 0.034068,0.009860,0.034102,0.819634])#更全匹配
# 计算欧几里得距离
distances = np.linalg.norm(data_r[['r_to_pc', 'r_to_nc', 'pr_to_pc', 'pr_to_nc', 'pr_P']].values - sample, axis=1)
#distances = np.linalg.norm(data_r[['好评无回应占比', '差评无回应占比', '价值共创占比', '评论端价值下降占比','回应端价值下降占比','价值共毁占比', 'pr_P']].values - sample, axis=1)
# 添加距离到DataFrame
data_r['distance'] = distances
# 排序并选择前10个最相似的样本
top_10_matches = data_r.sort_values(by='distance').head(100)
top_10_matches.describe()

,room_id,price,response_num,comment_num,positive_c_num,negtive_c_num,positive_r_num,negtive_r_num,co_create_num,c_decline_num,r_decline_num,co_destruction_num,positive_c_without_r,negtive_c_without_r,好评无回应占比,差评无回应占比,价值共创占比,评论端价值下降占比,回应端价值下降占比,价值共毁占比,r_to_pc,r_to_nc,pr_to_pc,pr_to_nc,pr_P,distance
count,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,2.544606e+07,5.695291,13.270000,81.410000,75.250000,5.870000,11.450000,1.690000,10.150000,1.270000,0.380000,1.300000,64.72000,3.300000,0.776590,0.047969,0.131228,0.019745,0.004522,0.019947,0.148920,0.467865,0.967265,0.495444,0.909114,0.246149
std,5.061866e+06,0.478809,12.310671,41.698556,40.960038,2.615629,11.867428,1.041512,11.761412,0.617178,0.762505,0.659047,37.90394,2.057261,0.115455,0.031292,0.105177,0.012793,0.008835,0.011966,0.117610,0.127291,0.072169,0.089606,0.047331,0.061842
min,1.368106e+07,4.709530,3.000000,20.000000,15.000000,3.000000,2.000000,1.000000,1.000000,1.000000,0.000000,1.000000,12.00000,1.000000,0.459184,0.009709,0.005000,0.004854,0.000000,0.004854,0.005236,0.166667,0.666667,0.250000,0.750000,0.095035
25%,2.167059e+07,5.392489,6.000000,50.000000,45.000000,4.000000,4.000000,1.000000,3.000000,1.000000,0.000000,1.000000,39.00000,2.000000,0.718241,0.025397,0.047013,0.010076,0.000000,0.010724,0.058541,0.375000,0.981513,0.500000,0.888365,0.195598
50%,2.582143e+07,5.666427,9.500000,75.000000,67.000000,5.000000,7.500000,1.000000,6.500000,1.000000,0.000000,1.000000,56.00000,3.000000,0.798307,0.039025,0.107746,0.016817,0.000000,0.016817,0.115282,0.500000,1.000000,0.500000,0.916165,0.255728
75%,2.947143e+07,5.988961,17.000000,105.000000,97.250000,7.000000,15.000000,2.000000,14.000000,1.000000,1.000000,1.000000,83.50000,4.000000,0.862461,0.064294,0.192857,0.025000,0.005788,0.025079,0.221760,0.513889,1.000000,0.500000,0.946344,0.298433
max,3.597522e+07,7.549609,81.000000,206.000000,202.000000,15.000000,77.000000,6.000000,76.000000,4.000000,4.000000,5.000000,190.00000,10.000000,0.969325,0.161290,0.418367,0.090909,0.039604,0.057143,0.476744,0.666667,1.000000,0.750000,0.980583,0.326867


In [187]:
#top_10_matches=data_r

In [188]:
# 计算Q1和Q3
Q1 = top_10_matches['price'].quantile(0.25)
Q3 = top_10_matches['price'].quantile(0.75)

IQR = Q3 - Q1
# 定义极端值界限
lower_bound = Q1- 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 过滤掉极端值
top_10_matches = top_10_matches[(top_10_matches['price'] >= lower_bound) & (top_10_matches['price'] <= upper_bound)]
top_10_matches.describe()

,room_id,price,response_num,comment_num,positive_c_num,negtive_c_num,positive_r_num,negtive_r_num,co_create_num,c_decline_num,r_decline_num,co_destruction_num,positive_c_without_r,negtive_c_without_r,好评无回应占比,差评无回应占比,价值共创占比,评论端价值下降占比,回应端价值下降占比,价值共毁占比,r_to_pc,r_to_nc,pr_to_pc,pr_to_nc,pr_P,distance
count,9.900000e+01,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
mean,2.544755e+07,5.676560,13.313131,81.979798,75.797980,5.888889,11.484848,1.696970,10.181818,1.272727,0.383838,1.303030,65.232323,3.313131,0.778778,0.047645,0.129726,0.019540,0.004567,0.019744,0.147057,0.467541,0.966934,0.495398,0.909812,0.246684
std,5.087604e+06,0.442896,12.365725,41.517633,40.798404,2.622076,11.922679,1.044466,11.816940,0.619713,0.765414,0.661701,37.747234,2.063514,0.113940,0.031283,0.104627,0.012692,0.008868,0.011853,0.116716,0.127897,0.072460,0.090061,0.047051,0.061924
min,1.368106e+07,4.709530,3.000000,20.000000,15.000000,3.000000,2.000000,1.000000,1.000000,1.000000,0.000000,1.000000,12.000000,1.000000,0.459184,0.009709,0.005000,0.004854,0.000000,0.004854,0.005236,0.166667,0.666667,0.250000,0.750000,0.095035
25%,2.163195e+07,5.391350,6.000000,50.500000,46.000000,4.000000,4.000000,1.000000,3.000000,1.000000,0.000000,1.000000,39.500000,2.000000,0.721111,0.025370,0.046967,0.010051,0.000000,0.010695,0.058258,0.375000,0.980268,0.500000,0.888889,0.196794
50%,2.601188e+07,5.666427,10.000000,75.000000,67.000000,5.000000,7.000000,1.000000,6.000000,1.000000,0.000000,1.000000,57.000000,3.000000,0.798450,0.038835,0.106796,0.016393,0.000000,0.016393,0.114286,0.500000,1.000000,0.500000,0.916667,0.256359
75%,2.947154e+07,5.986449,17.000000,106.000000,97.500000,7.000000,15.000000,2.000000,14.000000,1.000000,1.000000,1.000000,84.000000,4.000000,0.862853,0.062413,0.190175,0.025000,0.006142,0.025000,0.218634,0.527778,1.000000,0.500000,0.946452,0.298991
max,3.597522e+07,6.837333,81.000000,206.000000,202.000000,15.000000,77.000000,6.000000,76.000000,4.000000,4.000000,5.000000,190.000000,10.000000,0.969325,0.161290,0.418367,0.090909,0.039604,0.057143,0.476744,0.666667,1.000000,0.750000,0.980583,0.326867


In [189]:
# 计算均值和标准差
means = top_10_matches['price'].mean()
stds = top_10_matches['price'].std()

# 定义极端值的阈值
cutoff = stds * 3

# 计算每列的最小和最大有效值
lower_bounds = means - cutoff
upper_bounds = means + cutoff

# 使用条件筛选移除极端值
top_10_matches = top_10_matches[(top_10_matches['price'] > lower_bounds) & (top_10_matches['price'] < upper_bounds)]
top_10_matches.describe()

,room_id,price,response_num,comment_num,positive_c_num,negtive_c_num,positive_r_num,negtive_r_num,co_create_num,c_decline_num,r_decline_num,co_destruction_num,positive_c_without_r,negtive_c_without_r,好评无回应占比,差评无回应占比,价值共创占比,评论端价值下降占比,回应端价值下降占比,价值共毁占比,r_to_pc,r_to_nc,pr_to_pc,pr_to_nc,pr_P,distance
count,9.900000e+01,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
mean,2.544755e+07,5.676560,13.313131,81.979798,75.797980,5.888889,11.484848,1.696970,10.181818,1.272727,0.383838,1.303030,65.232323,3.313131,0.778778,0.047645,0.129726,0.019540,0.004567,0.019744,0.147057,0.467541,0.966934,0.495398,0.909812,0.246684
std,5.087604e+06,0.442896,12.365725,41.517633,40.798404,2.622076,11.922679,1.044466,11.816940,0.619713,0.765414,0.661701,37.747234,2.063514,0.113940,0.031283,0.104627,0.012692,0.008868,0.011853,0.116716,0.127897,0.072460,0.090061,0.047051,0.061924
min,1.368106e+07,4.709530,3.000000,20.000000,15.000000,3.000000,2.000000,1.000000,1.000000,1.000000,0.000000,1.000000,12.000000,1.000000,0.459184,0.009709,0.005000,0.004854,0.000000,0.004854,0.005236,0.166667,0.666667,0.250000,0.750000,0.095035
25%,2.163195e+07,5.391350,6.000000,50.500000,46.000000,4.000000,4.000000,1.000000,3.000000,1.000000,0.000000,1.000000,39.500000,2.000000,0.721111,0.025370,0.046967,0.010051,0.000000,0.010695,0.058258,0.375000,0.980268,0.500000,0.888889,0.196794
50%,2.601188e+07,5.666427,10.000000,75.000000,67.000000,5.000000,7.000000,1.000000,6.000000,1.000000,0.000000,1.000000,57.000000,3.000000,0.798450,0.038835,0.106796,0.016393,0.000000,0.016393,0.114286,0.500000,1.000000,0.500000,0.916667,0.256359
75%,2.947154e+07,5.986449,17.000000,106.000000,97.500000,7.000000,15.000000,2.000000,14.000000,1.000000,1.000000,1.000000,84.000000,4.000000,0.862853,0.062413,0.190175,0.025000,0.006142,0.025000,0.218634,0.527778,1.000000,0.500000,0.946452,0.298991
max,3.597522e+07,6.837333,81.000000,206.000000,202.000000,15.000000,77.000000,6.000000,76.000000,4.000000,4.000000,5.000000,190.000000,10.000000,0.969325,0.161290,0.418367,0.090909,0.039604,0.057143,0.476744,0.666667,1.000000,0.750000,0.980583,0.326867


In [190]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
minmax_scaler = MinMaxScaler()
top_10_matches['price'] = minmax_scaler.fit_transform(top_10_matches[['price']])
top_10_matches.describe()

,room_id,price,response_num,comment_num,positive_c_num,negtive_c_num,positive_r_num,negtive_r_num,co_create_num,c_decline_num,r_decline_num,co_destruction_num,positive_c_without_r,negtive_c_without_r,好评无回应占比,差评无回应占比,价值共创占比,评论端价值下降占比,回应端价值下降占比,价值共毁占比,r_to_pc,r_to_nc,pr_to_pc,pr_to_nc,pr_P,distance
count,9.900000e+01,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
mean,2.544755e+07,0.454474,13.313131,81.979798,75.797980,5.888889,11.484848,1.696970,10.181818,1.272727,0.383838,1.303030,65.232323,3.313131,0.778778,0.047645,0.129726,0.019540,0.004567,0.019744,0.147057,0.467541,0.966934,0.495398,0.909812,0.246684
std,5.087604e+06,0.208147,12.365725,41.517633,40.798404,2.622076,11.922679,1.044466,11.816940,0.619713,0.765414,0.661701,37.747234,2.063514,0.113940,0.031283,0.104627,0.012692,0.008868,0.011853,0.116716,0.127897,0.072460,0.090061,0.047051,0.061924
min,1.368106e+07,0.000000,3.000000,20.000000,15.000000,3.000000,2.000000,1.000000,1.000000,1.000000,0.000000,1.000000,12.000000,1.000000,0.459184,0.009709,0.005000,0.004854,0.000000,0.004854,0.005236,0.166667,0.666667,0.250000,0.750000,0.095035
25%,2.163195e+07,0.320434,6.000000,50.500000,46.000000,4.000000,4.000000,1.000000,3.000000,1.000000,0.000000,1.000000,39.500000,2.000000,0.721111,0.025370,0.046967,0.010051,0.000000,0.010695,0.058258,0.375000,0.980268,0.500000,0.888889,0.196794
50%,2.601188e+07,0.449711,10.000000,75.000000,67.000000,5.000000,7.000000,1.000000,6.000000,1.000000,0.000000,1.000000,57.000000,3.000000,0.798450,0.038835,0.106796,0.016393,0.000000,0.016393,0.114286,0.500000,1.000000,0.500000,0.916667,0.256359
75%,2.947154e+07,0.600111,17.000000,106.000000,97.500000,7.000000,15.000000,2.000000,14.000000,1.000000,1.000000,1.000000,84.000000,4.000000,0.862853,0.062413,0.190175,0.025000,0.006142,0.025000,0.218634,0.527778,1.000000,0.500000,0.946452,0.298991
max,3.597522e+07,1.000000,81.000000,206.000000,202.000000,15.000000,77.000000,6.000000,76.000000,4.000000,4.000000,5.000000,190.000000,10.000000,0.969325,0.161290,0.418367,0.090909,0.039604,0.057143,0.476744,0.666667,1.000000,0.750000,0.980583,0.326867


In [191]:
minmax_scaler = MinMaxScaler()
data['ARPP1'] = minmax_scaler.fit_transform(data[['ARPP1']])
data['ARSP1'] = minmax_scaler.fit_transform(data[['ARSP1']])
data.describe()

,step,提供方1类型,提供方2类型,提供方1利他程度,提供方2利他程度,OP1,OP2,PCIP1,PCIP2,ARPP1,ARPP2,ARSP1,ARSP2,服务人数1,服务人数2,总服务人数,社会总福利,平台收益,提供方收益1,提供方收益2,提供方总收益,消费者收益1,消费者收益2,消费者总收益,实际到达人数,办理入住数1,办理入住数2,办理入住总数,被激励好评数量1,被激励好评数量2,获得激励数量1,获得激励数量2,好评占比1,好评占比2,总好评数量占比,PERP1,PERP2,PCIP,OP,PERP,ARPP,ARSP,平均毛利率,平均净利率,毛利率1,毛利率2,净利率1,净利率2,消费者平均收益1,消费者平均收益2,总评论数量1,总评论数量2,总评论数量,好评数量1,好评数量2,好评总数,当期好评占比1,当期好评占比2,被激励好评占好评比例1,被激励好评占好评比例2,被激励好评占好评比例,获得激励评论数量占评论比例1,获得激励评论数量占评论比例2,获得激励评论数量占评论比例,RP1,RP2,val1,expect1,val2,expect2,累计评论数量1,累计评论数量2,好评无回应占比1,差评无回应占比1,价值共创占比1,评论端价值下降占比1,回应端价值下降占比1,价值共毁占比1,好评无回应占比2,差评无回应占比2,价值共创占比2,评论端价值下降占比2,回应端价值下降占比2,价值共毁占比2,对好评回应率1,对差评回应率1,对好评提升型回应率1,对差评提升型回应率1,累计好评占比1
count,198.00000,198.0,198.0,198.0,198.0,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.0,198.0,198.000000,198.000000,198.0,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000
mean,51.00000,0.0,0.0,0.0,0.0,0.153510,0.153510,0.079916,0.079916,0.494015,0.568730,0.443651,0.246607,121.004914,121.004914,242.009828,254.278426,63.776107,72.824715,72.824715,145.649430,22.426445,22.426445,44.852889,281.267657,120.990247,120.990247,241.980495,61.393970,61.393970,80.067429,80.067429,0.820390,0.820390,0.819528,0.5,0.5,0.079916,0.153510,0.5,0.568730,0.246607,0.322123,0.236814,0.322123,0.322123,0.236814,0.236814,0.185806,0.185806,98.024616,98.024616,196.049232,80.067429,80.067429,160.134859,0.819821,0.819821,0.756470,0.756470,0.756470,0.819821,0.819821,0.819821,0.738890,0.738890,-0.022123,0.012797,-0.022123,0.012797,4990.947591,4990.947591,0.657319,0.112195,0.152456,0.034068,0.009860,0.034102,0.657319,0.112195,0.152456,0.034068,0.009860,0.034102,0.198034,0.377884,0.939258,0.499690,0.819634
std,28.64982,0.0,0.0,0.0,0.0,0.044272,0.044272,0.025136,0.025136,0.231269,0.109374,0.214618,0.101429,7.106736,7.106736,6.158345,6.767688,8.830153,8.153248,8.153248,9.871847,6.350368,6.350368,9.655781,0.812902,8.147983,8.147983,6.783515,19.745339,19.745339,13.267594,13.267594,0.024864,0.024864,0.020736,0.0,0.0,0.017080,0.031890,0.0,0.072977,0.071106,0.013326,0.015177,0.021888,0.021888,0.020620,0.020620,0.052817,0.052817,11.038079,11.038079,15.812260,13.267594,13.267594,19.339440,0.115344,0.115344,0.166088,0.166088,0.119919,0.115344,0.115344,0.088433,0.070069,0.070069,0.021888,0.017766,0.021888,0.017766,2817.108295,2817.108295,0.019306,0.014958,0.004494,0.004585,0.000309,0.004555,0.019306,0.014958,0.004494,0.004585,0.000309,0.004555,0.000262,0.001153,0.000361,0.000880,0.024096
min,2.00000,0.0,0.0,0.0,0.0,0.051155,0.051155,0.020687,0.020687,0.000000,0.335096,0.000000,0.036937,102.196000,102.196000,229.991000,237.310142,46.020960,56.001098,56.001098,124.266783,8.439699,8.439699,27.938355,278.572000,99.011000,99.011000,228.773000,12.758000,12.758000,41.385000,41.385000,0.782574,0.782574,0.801876,0.5,0.5,0.041166,0.073047,0.5,0.386208,0.073369,0.271388,0.194806,0.263574,0.263574,0.172255,0.172255,0.063132,0.063132,72.330000,72.330000,165.570000,41.385000,41.385000,115.326000,0.391022,0.391022,0.190985,0.190985,0.419832,0.391022,0.391022,0.547487,0.636857,0.636857,-0.102230,-0.071345,-0.102230,-0.071345,168.090000,168.090000,0.626555,0.036409,0.145369,0.010607,0.009314,0.010596,0.626555,0.036409,0.145369,0.010607,0.009314,0.010596,0.197521,0.368030,0.937388

In [192]:
plt.rcParams['font.sans-serif'] = ['Times New Roman']
from scipy.stats import gaussian_kde
font_size=13
title_size=13
def kde_estimate(data_test):
    # 创建核密度估计对象
    kde = gaussian_kde(data_test)
    # 生成一系列用于绘制核密度估计曲线的点
    x = np.linspace(min(data_test), max(data_test), 1000)
    # 计算核密度估计值
    kde_values = kde(x)
    return x,kde_values

fig=plt.figure(figsize=(12,6))
columns1=['price']
columns2=['ARPP1']
columns3=['(a) Kernel Density Estimation of Provider Prices\n(Log-Transformed Prices)']
columns4=['Provider Prices (Normalized)']
for i in range(1):   
    ax = fig.add_subplot(1,2,i+1)
    data_test = top_10_matches[columns1[i]].dropna()
    x,kde_values=kde_estimate(data_test)
    # 绘制核密度估计图
    ax.plot(x, kde_values, label='Real data')
    ax.fill_between(x, kde_values, color="skyblue", alpha=0.5)
    data_test = data[columns2[i]].dropna()
    x,kde_values=kde_estimate(data_test)
    ax.plot(x, kde_values, label='Simulation')
    ax.fill_between(x, kde_values, color="orange", alpha=0.3)
    #ax.set_xlabel(columns4[i],size=font_size)
    if i==0:
        ax.set_ylabel('Density',size=font_size)
    ax.set_title(columns3[i],fontsize=title_size,y=-0.18)
    ax.legend(loc='best', fontsize=font_size)
    ax.tick_params(axis='both', labelsize=font_size)
#fig.subplots_adjust(bottom=0.15,wspace=0.08)#子图间距
#plt.show()

In [193]:
from scipy.stats import ks_2samp
# 统计检验：Kolmogorov-Smirnov检验-检验两样本是否来自相同的连续分布
ks_statistic, p_value = ks_2samp(data['ARSP1'].dropna(), top_10_matches['price'].dropna())
print("K-S统计量:", ks_statistic)
print("p值:", p_value)

alpha = 0.05
if p_value > alpha:
    print("p值大于显著性水平，无法拒绝两个样本来自相同分布的假设。")
else:
    print("p值小于显著性水平，拒绝两个样本来自相同分布的假设。")

K-S统计量: 0.1111111111111111
p值: 0.380855492766882
p值大于显著性水平，无法拒绝两个样本来自相同分布的假设。


###### 方法二：针对于现实数据中价格呈现的正拖尾现象，本研究截取前80%的提供方，以探讨仿真模型中的提供方与最普遍的这群提供方之间定价的一致性。（收取高昂价格的更可能是连锁酒店或星级酒店或成本极高可能存在其他效应吸引消费者。）

In [177]:
data_r=pd.read_csv(r'data_merge_all2.csv')
data_r=data_r[['room_id','price']]
data_r=data_r.groupby('room_id').mean().reset_index()
data_r = pd.merge(data_r, gb, on='room_id', how='inner')

In [178]:
sample = np.array([0.198034, 0.377884, 0.939258, 0.499690,0.819634])
#sample = np.array([0.198034, 0.377884, 0.939258, 0.499690,0.657319, 0.112195, 0.152456, 0.034068,0.009860,0.034102,0.819634])#更全匹配
# 计算欧几里得距离
distances = np.linalg.norm(data_r[['r_to_pc', 'r_to_nc', 'pr_to_pc', 'pr_to_nc', 'pr_P']].values - sample, axis=1)
#distances = np.linalg.norm(data_r[['r_to_pc', 'r_to_nc', 'pr_to_pc', 'pr_to_nc','好评无回应占比', '差评无回应占比', '价值共创占比', '评论端价值下降占比','回应端价值下降占比','价值共毁占比', 'pr_P']].values - sample, axis=1)
# 添加距离到DataFrame
data_r['distance'] = distances
# 排序并选择前10个最相似的样本
top_10_matches = data_r.sort_values(by='distance').head(100)
top_10_matches.describe()

,room_id,price,response_num,comment_num,positive_c_num,negtive_c_num,positive_r_num,negtive_r_num,co_create_num,c_decline_num,r_decline_num,co_destruction_num,positive_c_without_r,negtive_c_without_r,好评无回应占比,差评无回应占比,价值共创占比,评论端价值下降占比,回应端价值下降占比,价值共毁占比,r_to_pc,r_to_nc,pr_to_pc,pr_to_nc,pr_P,distance
count,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,2.544606e+07,336.840000,13.270000,81.410000,75.250000,5.870000,11.450000,1.690000,10.150000,1.270000,0.380000,1.300000,64.72000,3.300000,0.776590,0.047969,0.131228,0.019745,0.004522,0.019947,0.148920,0.467865,0.967265,0.495444,0.909114,0.246149
std,5.061866e+06,217.681062,12.310671,41.698556,40.960038,2.615629,11.867428,1.041512,11.761412,0.617178,0.762505,0.659047,37.90394,2.057261,0.115455,0.031292,0.105177,0.012793,0.008835,0.011966,0.117610,0.127291,0.072169,0.089606,0.047331,0.061842
min,1.368106e+07,110.000000,3.000000,20.000000,15.000000,3.000000,2.000000,1.000000,1.000000,1.000000,0.000000,1.000000,12.00000,1.000000,0.459184,0.009709,0.005000,0.004854,0.000000,0.004854,0.005236,0.166667,0.666667,0.250000,0.750000,0.095035
25%,2.167059e+07,218.750000,6.000000,50.000000,45.000000,4.000000,4.000000,1.000000,3.000000,1.000000,0.000000,1.000000,39.00000,2.000000,0.718241,0.025397,0.047013,0.010076,0.000000,0.010724,0.058541,0.375000,0.981513,0.500000,0.888365,0.195598
50%,2.582143e+07,288.000000,9.500000,75.000000,67.000000,5.000000,7.500000,1.000000,6.500000,1.000000,0.000000,1.000000,56.00000,3.000000,0.798307,0.039025,0.107746,0.016817,0.000000,0.016817,0.115282,0.500000,1.000000,0.500000,0.916165,0.255728
75%,2.947143e+07,398.000000,17.000000,105.000000,97.250000,7.000000,15.000000,2.000000,14.000000,1.000000,1.000000,1.000000,83.50000,4.000000,0.862461,0.064294,0.192857,0.025000,0.005788,0.025079,0.221760,0.513889,1.000000,0.500000,0.946344,0.298433
max,3.597522e+07,1899.000000,81.000000,206.000000,202.000000,15.000000,77.000000,6.000000,76.000000,4.000000,4.000000,5.000000,190.00000,10.000000,0.969325,0.161290,0.418367,0.090909,0.039604,0.057143,0.476744,0.666667,1.000000,0.750000,0.980583,0.326867


In [179]:
#top_10_matches=data_r

In [180]:
# 计算Q1和Q3
Q1 = top_10_matches['price'].quantile(0)
Q3 = top_10_matches['price'].quantile(0.8)

# 定义极端值界限
lower_bound = Q1
upper_bound = Q3

# 过滤掉极端值
top_10_matches = top_10_matches[(top_10_matches['price'] >= lower_bound) & (top_10_matches['price'] <= upper_bound)]
top_10_matches.describe()

,room_id,price,response_num,comment_num,positive_c_num,negtive_c_num,positive_r_num,negtive_r_num,co_create_num,c_decline_num,r_decline_num,co_destruction_num,positive_c_without_r,negtive_c_without_r,好评无回应占比,差评无回应占比,价值共创占比,评论端价值下降占比,回应端价值下降占比,价值共毁占比,r_to_pc,r_to_nc,pr_to_pc,pr_to_nc,pr_P,distance
count,8.000000e+01,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.00000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000
mean,2.606118e+07,261.937500,12.837500,82.525000,76.287500,5.925000,11.012500,1.700000,9.737500,1.237500,0.40000,1.287500,66.150000,3.400000,0.779448,0.049156,0.127445,0.019635,0.004626,0.019690,0.145272,0.461280,0.966517,0.494931,0.907918,0.249581
std,4.920160e+06,80.177389,12.569245,43.636002,43.023159,2.754858,12.115994,1.118317,12.057608,0.579038,0.82062,0.697078,40.440643,2.208155,0.121342,0.033185,0.108664,0.013440,0.009183,0.012190,0.122017,0.135124,0.074495,0.085205,0.049362,0.061996
min,1.518955e+07,110.000000,3.000000,20.000000,15.000000,3.000000,2.000000,1.000000,1.000000,1.000000,0.00000,1.000000,12.000000,1.000000,0.459184,0.009709,0.005000,0.004854,0.000000,0.004854,0.005236,0.166667,0.666667,0.250000,0.750000,0.095035
25%,2.188732e+07,208.000000,5.000000,49.250000,42.500000,4.000000,4.000000,1.000000,3.000000,1.000000,0.00000,1.000000,38.000000,2.000000,0.709722,0.024245,0.039537,0.009853,0.000000,0.010051,0.042354,0.333333,0.995690,0.500000,0.885002,0.197374
50%,2.685204e+07,250.000000,8.500000,75.500000,67.000000,5.000000,7.000000,1.000000,6.000000,1.000000,0.00000,1.000000,56.000000,3.000000,0.802778,0.039737,0.098285,0.016001,0.000000,0.016261,0.106289,0.500000,1.000000,0.500000,0.915278,0.265089
75%,2.983881e+07,318.250000,14.250000,108.250000,97.500000,7.000000,13.250000,2.000000,12.250000,1.000000,0.25000,1.000000,85.250000,4.000000,0.873230,0.064294,0.190024,0.025329,0.001359,0.025000,0.221760,0.513889,1.000000,0.500000,0.948386,0.302661
max,3.597522e+07,418.000000,81.000000,206.000000,202.000000,15.000000,77.000000,6.000000,76.000000,4.000000,4.00000,5.000000,190.000000,10.000000,0.969325,0.161290,0.418367,0.090909,0.039604,0.057143,0.476744,0.666667,1.000000,0.666667,0.980583,0.326867


In [181]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
minmax_scaler = MinMaxScaler()
top_10_matches['price'] = minmax_scaler.fit_transform(top_10_matches[['price']])
top_10_matches.describe()

,room_id,price,response_num,comment_num,positive_c_num,negtive_c_num,positive_r_num,negtive_r_num,co_create_num,c_decline_num,r_decline_num,co_destruction_num,positive_c_without_r,negtive_c_without_r,好评无回应占比,差评无回应占比,价值共创占比,评论端价值下降占比,回应端价值下降占比,价值共毁占比,r_to_pc,r_to_nc,pr_to_pc,pr_to_nc,pr_P,distance
count,8.000000e+01,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.00000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000
mean,2.606118e+07,0.493304,12.837500,82.525000,76.287500,5.925000,11.012500,1.700000,9.737500,1.237500,0.40000,1.287500,66.150000,3.400000,0.779448,0.049156,0.127445,0.019635,0.004626,0.019690,0.145272,0.461280,0.966517,0.494931,0.907918,0.249581
std,4.920160e+06,0.260316,12.569245,43.636002,43.023159,2.754858,12.115994,1.118317,12.057608,0.579038,0.82062,0.697078,40.440643,2.208155,0.121342,0.033185,0.108664,0.013440,0.009183,0.012190,0.122017,0.135124,0.074495,0.085205,0.049362,0.061996
min,1.518955e+07,0.000000,3.000000,20.000000,15.000000,3.000000,2.000000,1.000000,1.000000,1.000000,0.00000,1.000000,12.000000,1.000000,0.459184,0.009709,0.005000,0.004854,0.000000,0.004854,0.005236,0.166667,0.666667,0.250000,0.750000,0.095035
25%,2.188732e+07,0.318182,5.000000,49.250000,42.500000,4.000000,4.000000,1.000000,3.000000,1.000000,0.00000,1.000000,38.000000,2.000000,0.709722,0.024245,0.039537,0.009853,0.000000,0.010051,0.042354,0.333333,0.995690,0.500000,0.885002,0.197374
50%,2.685204e+07,0.454545,8.500000,75.500000,67.000000,5.000000,7.000000,1.000000,6.000000,1.000000,0.00000,1.000000,56.000000,3.000000,0.802778,0.039737,0.098285,0.016001,0.000000,0.016261,0.106289,0.500000,1.000000,0.500000,0.915278,0.265089
75%,2.983881e+07,0.676136,14.250000,108.250000,97.500000,7.000000,13.250000,2.000000,12.250000,1.000000,0.25000,1.000000,85.250000,4.000000,0.873230,0.064294,0.190024,0.025329,0.001359,0.025000,0.221760,0.513889,1.000000,0.500000,0.948386,0.302661
max,3.597522e+07,1.000000,81.000000,206.000000,202.000000,15.000000,77.000000,6.000000,76.000000,4.000000,4.00000,5.000000,190.000000,10.000000,0.969325,0.161290,0.418367,0.090909,0.039604,0.057143,0.476744,0.666667,1.000000,0.666667,0.980583,0.326867


In [182]:
minmax_scaler = MinMaxScaler()
data['ARPP1'] = minmax_scaler.fit_transform(data[['ARPP1']])
data['ARSP1'] = minmax_scaler.fit_transform(data[['ARSP1']])
data.describe()

,step,提供方1类型,提供方2类型,提供方1利他程度,提供方2利他程度,OP1,OP2,PCIP1,PCIP2,ARPP1,ARPP2,ARSP1,ARSP2,服务人数1,服务人数2,总服务人数,社会总福利,平台收益,提供方收益1,提供方收益2,提供方总收益,消费者收益1,消费者收益2,消费者总收益,实际到达人数,办理入住数1,办理入住数2,办理入住总数,被激励好评数量1,被激励好评数量2,获得激励数量1,获得激励数量2,好评占比1,好评占比2,总好评数量占比,PERP1,PERP2,PCIP,OP,PERP,ARPP,ARSP,平均毛利率,平均净利率,毛利率1,毛利率2,净利率1,净利率2,消费者平均收益1,消费者平均收益2,总评论数量1,总评论数量2,总评论数量,好评数量1,好评数量2,好评总数,当期好评占比1,当期好评占比2,被激励好评占好评比例1,被激励好评占好评比例2,被激励好评占好评比例,获得激励评论数量占评论比例1,获得激励评论数量占评论比例2,获得激励评论数量占评论比例,RP1,RP2,val1,expect1,val2,expect2,累计评论数量1,累计评论数量2,好评无回应占比1,差评无回应占比1,价值共创占比1,评论端价值下降占比1,回应端价值下降占比1,价值共毁占比1,好评无回应占比2,差评无回应占比2,价值共创占比2,评论端价值下降占比2,回应端价值下降占比2,价值共毁占比2,对好评回应率1,对差评回应率1,对好评提升型回应率1,对差评提升型回应率1,累计好评占比1
count,198.00000,198.0,198.0,198.0,198.0,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.0,198.0,198.000000,198.000000,198.0,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000
mean,51.00000,0.0,0.0,0.0,0.0,0.153510,0.153510,0.079916,0.079916,0.494015,0.568730,0.443651,0.246607,121.004914,121.004914,242.009828,254.278426,63.776107,72.824715,72.824715,145.649430,22.426445,22.426445,44.852889,281.267657,120.990247,120.990247,241.980495,61.393970,61.393970,80.067429,80.067429,0.820390,0.820390,0.819528,0.5,0.5,0.079916,0.153510,0.5,0.568730,0.246607,0.322123,0.236814,0.322123,0.322123,0.236814,0.236814,0.185806,0.185806,98.024616,98.024616,196.049232,80.067429,80.067429,160.134859,0.819821,0.819821,0.756470,0.756470,0.756470,0.819821,0.819821,0.819821,0.738890,0.738890,-0.022123,0.012797,-0.022123,0.012797,4990.947591,4990.947591,0.657319,0.112195,0.152456,0.034068,0.009860,0.034102,0.657319,0.112195,0.152456,0.034068,0.009860,0.034102,0.198034,0.377884,0.939258,0.499690,0.819634
std,28.64982,0.0,0.0,0.0,0.0,0.044272,0.044272,0.025136,0.025136,0.231269,0.109374,0.214618,0.101429,7.106736,7.106736,6.158345,6.767688,8.830153,8.153248,8.153248,9.871847,6.350368,6.350368,9.655781,0.812902,8.147983,8.147983,6.783515,19.745339,19.745339,13.267594,13.267594,0.024864,0.024864,0.020736,0.0,0.0,0.017080,0.031890,0.0,0.072977,0.071106,0.013326,0.015177,0.021888,0.021888,0.020620,0.020620,0.052817,0.052817,11.038079,11.038079,15.812260,13.267594,13.267594,19.339440,0.115344,0.115344,0.166088,0.166088,0.119919,0.115344,0.115344,0.088433,0.070069,0.070069,0.021888,0.017766,0.021888,0.017766,2817.108295,2817.108295,0.019306,0.014958,0.004494,0.004585,0.000309,0.004555,0.019306,0.014958,0.004494,0.004585,0.000309,0.004555,0.000262,0.001153,0.000361,0.000880,0.024096
min,2.00000,0.0,0.0,0.0,0.0,0.051155,0.051155,0.020687,0.020687,0.000000,0.335096,0.000000,0.036937,102.196000,102.196000,229.991000,237.310142,46.020960,56.001098,56.001098,124.266783,8.439699,8.439699,27.938355,278.572000,99.011000,99.011000,228.773000,12.758000,12.758000,41.385000,41.385000,0.782574,0.782574,0.801876,0.5,0.5,0.041166,0.073047,0.5,0.386208,0.073369,0.271388,0.194806,0.263574,0.263574,0.172255,0.172255,0.063132,0.063132,72.330000,72.330000,165.570000,41.385000,41.385000,115.326000,0.391022,0.391022,0.190985,0.190985,0.419832,0.391022,0.391022,0.547487,0.636857,0.636857,-0.102230,-0.071345,-0.102230,-0.071345,168.090000,168.090000,0.626555,0.036409,0.145369,0.010607,0.009314,0.010596,0.626555,0.036409,0.145369,0.010607,0.009314,0.010596,0.197521,0.368030,0.937388

In [183]:
from scipy.stats import gaussian_kde
font_size=13
title_size=13
def kde_estimate(data_test):
    # 创建核密度估计对象
    kde = gaussian_kde(data_test)
    # 生成一系列用于绘制核密度估计曲线的点
    x = np.linspace(min(data_test), max(data_test), 1000)
    # 计算核密度估计值
    kde_values = kde(x)
    return x,kde_values

#fig=plt.figure(figsize=(6,6))
columns1=['price']
columns2=['ARPP1']
columns3=['(b) Kernel Density Estimation of Provider Prices\n(Top 80% of Prices)']
columns4=['Provider Prices (Normalized)']
for i in range(1):   
    ax = fig.add_subplot(1,2,i+2)
    data_test = top_10_matches[columns1[i]].dropna()
    x,kde_values=kde_estimate(data_test)
    # 绘制核密度估计图
    ax.plot(x, kde_values, label='Real data')
    ax.fill_between(x, kde_values, color="skyblue", alpha=0.5)
    data_test = data[columns2[i]].dropna()
    x,kde_values=kde_estimate(data_test)
    ax.plot(x, kde_values, label='Simulation')
    ax.fill_between(x, kde_values, color="orange", alpha=0.3)
    #ax.set_xlabel(columns4[i],size=font_size)
    #if i==0:
    #    ax.set_ylabel('Density',size=font_size)
    ax.set_title(columns3[i],fontsize=title_size,y=-0.18)
    ax.legend(loc='best', fontsize=font_size)
    ax.tick_params(axis='both', labelsize=font_size)
fig.subplots_adjust(bottom=0.15,wspace=0.15)#子图间距
plt.show()

In [297]:
#绘制累计分布图
fig=plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
data_test = top_10_matches[columns1[i]].dropna()
x,kde_values=kde_estimate(data_test)
cdf_values = np.cumsum(kde_values) * (x[1] - x[0])# 计算CDF
ax.plot(x, cdf_values, label='Real data')
ax.fill_between(x, cdf_values, color="skyblue", alpha=0.5)
data_test = data[columns2[i]].dropna()
x,kde_values=kde_estimate(data_test)
cdf_values = np.cumsum(kde_values) * (x[1] - x[0])# 计算CDF
ax.plot(x, cdf_values, label='Real data')
ax.fill_between(x, cdf_values, color="skyblue", alpha=0.5)
ax.set_xlabel('Merchant Price (Normalized)',size=font_size)
ax.set_ylabel('Cumulative Probability',size=font_size)
ax.set_title('Cumulative Distribution Function',fontsize=title_size,y=-0.2)
ax.legend(loc='best', fontsize=font_size)
ax.tick_params(axis='both', labelsize=font_size)
plt.show()

In [184]:
from scipy.stats import ks_2samp
# 统计检验：Kolmogorov-Smirnov检验-检验两样本是否来自相同的连续分布
ks_statistic, p_value = ks_2samp(data['ARSP1'].dropna(), top_10_matches['price'].dropna())
print("K-S统计量:", ks_statistic)
print("p值:", p_value)

alpha = 0.05
if p_value > alpha:
    print("p值大于显著性水平，无法拒绝两个样本来自相同分布的假设。")
else:
    print("p值小于显著性水平，拒绝两个样本来自相同分布的假设。")

K-S统计量: 0.12209595959595959
p值: 0.33046976300211434
p值大于显著性水平，无法拒绝两个样本来自相同分布的假设。


##### 仿真结果中到达人数分布

In [620]:
x,kde_values=kde_estimate(data['办理入住总数'])
plt.plot(x, kde_values, label='办理入住总数')
plt.fill_between(x, kde_values, color="skyblue", alpha=0.5)
# 绘图
plt.title('Kernel Density Plot')
plt.ylabel('Density')
plt.legend(loc='best', fontsize=font_size)
plt.show()

# 评论对销量和未来评论量的影响

In [6]:
def review_count(row):
    row['评论1']=[float(x) for x in row['评论1'].strip('[]').split()]
    row['评论2']=[float(x) for x in row['评论2'].strip('[]').split()]
    return row

def review_compute1(row):
    if row['评论1_lag1']:
        return list(np.array(row['评论1'])-np.array(row['评论1_lag1']))
    else:
        return row['评论1']
               
def review_compute2(row):
    if row['评论2_lag1']:
        return list(np.array(row['评论2'])-np.array(row['评论2_lag1']))
    else:
        return row['评论2']

def create_lag_column(group):
    group['评论1_lag1'] = group['评论1'].shift(1)
    group['评论2_lag1'] = group['评论2'].shift(1)
    group.loc[1:,['评论1_当期']]=group.loc[1:,['评论1','评论1_lag1']].apply(review_compute1,axis=1)
    group.loc[1:,['评论2_当期']]=group.loc[1:,['评论2','评论2_lag1']].apply(review_compute2,axis=1)
    return group

def review_per(row):
    b_weight=10
    b=b_weight*np.array([0.11,-0.15,0.17,0,0,-0.23])
    if row['评论1_lag1']:
        row['评论感知1']=np.dot(np.array(row['评论1_lag1'])/(sum(np.array(row['评论1_lag1']))), b.T)
    else:
        row['评论感知1']=0
        
    if row['评论2_lag1']:
        row['评论感知2']=np.dot(np.array(row['评论2_lag1'])/(sum(np.array(row['评论2_lag1']))), b.T)
    else:
        row['评论感知2']=0
        
    row['累计评论数量1']=sum(row['评论1'])
    row['累计评论数量2']=sum(row['评论2'])
    row['当期评论数量1']=sum(row['评论1_当期'])
    row['当期评论数量2']=sum(row['评论2_当期'])
    row['好评无回应占比1']=row['评论1'][0]/(1+row['累计评论数量1'])
    row['差评无回应占比1']=row['评论1'][1]/(1+row['累计评论数量1'])
    row['价值共创占比1']=row['评论1'][2]/(1+row['累计评论数量1'])
    row['评论端价值下降占比1']=row['评论1'][3]/(1+row['累计评论数量1'])
    row['回应端价值下降占比1']=row['评论1'][4]/(1+row['累计评论数量1'])
    row['价值共毁占比1']=row['评论1'][5]/(1+row['累计评论数量1'])
    row['好评无回应占比2']=row['评论2'][0]/(1+row['累计评论数量2'])
    row['差评无回应占比2']=row['评论2'][1]/(1+row['累计评论数量2'])
    row['价值共创占比2']=row['评论2'][2]/(1+row['累计评论数量2'])
    row['评论端价值下降占比2']=row['评论2'][3]/(1+row['累计评论数量2'])
    row['回应端价值下降占比2']=row['评论2'][4]/(1+row['累计评论数量2'])
    row['价值共毁占比2']=row['评论2'][5]/(1+row['累计评论数量2'])
    return row

def review_num(row):
    if row.name%1000==0:
        print(row.name)
    row['评论1']=[float(x) for x in row['评论1'].strip('[]').split(', ')]
    row['评论2']=[float(x) for x in row['评论2'].strip('[]').split(', ')]
    row['好评无回应1']=row['评论1'][0]
    row['差评无回应1']=row['评论1'][1]
    row['价值共创1']=row['评论1'][2]
    row['评论端价值下降1']=row['评论1'][3]
    row['回应端价值下降1']=row['评论1'][4]
    row['价值共毁1']=row['评论1'][5]
    row['好评无回应2']=row['评论2'][0]
    row['差评无回应2']=row['评论2'][1]
    row['价值共创2']=row['评论2'][2]
    row['评论端价值下降2']=row['评论2'][3]
    row['回应端价值下降2']=row['评论2'][4]
    row['价值共毁2']=row['评论2'][5]
    row['累计好评1']=row['评论1'][0]+row['评论1'][2]+row['评论1'][4]
    return row

from statsmodels.stats.outliers_influence import variance_inflation_factor
def calculate_vif(df, columns):
    # Initialize DataFrame to store VIF scores
    vif_data = pd.DataFrame()
    vif_data["feature"] = columns

    # Calculate VIF for each feature
    try:
        # Collects each column's VIF score
        vif_data["VIF"] = [variance_inflation_factor(df[columns].values, i) for i in range(len(columns))]
    except Exception as e:
        print(f"An error occurred: {e}")
        print(columns)
    return vif_data

In [7]:
data=pd.read_csv(r'C:\Users\付应超\Desktop\小论文数据-在线评论\数据-jais修改\数据\data_all.csv')
data.loc[(data['提供方1类型']==0) &(data['提供方2类型']==0),['market_type']]='pure-egoism'
data.loc[(data['提供方1类型']==0) &(data['提供方2类型']==1),['market_type']]='hybrid'
data.loc[(data['提供方1类型']==1) &(data['提供方2类型']==1),['market_type']]='pure-altruism'
data=data[(data['market_type']=='pure-egoism') & (data['回应偏好1']==0.5) & (data['回应偏好2']==0.5)]
data=data.apply(review_count,axis=1)
data=data.groupby(['提供方1类型', '提供方2类型','回应偏好1', '回应偏好2']).apply(create_lag_column)
data.reset_index(drop=True, inplace=True)
data['评论1']=data['评论1_lag1']
data['评论2']=data['评论2_lag1']
data=data[data['step']>1]
data=data.apply(review_per,axis=1)
data=data.apply(review_num,axis=1)
data=data.rename(columns={'夸大宣传程度1':'OP1','夸大宣传程度2':'OP2','好评激励1':'PCIP1','好评激励2':'PCIP2',
                '回应偏好1':'PERP1','回应偏好2':'PERP2','价格1':'ARPP1','价格2':'ARPP2','服务质量1':'ARSP1','服务质量2':'ARSP2',
                '平均夸大宣传程度':'OP','平均好评激励金额':'PCIP','平均回应偏好':'PERP','平均价格':'ARPP','平均服务质量':'ARSP',
                '评论感知1':'RP1','评论感知2':'RP2'})

提供方1类型         0.0
提供方2类型         0.0
回应偏好1          0.1
回应偏好2          0.1
random_seed    1.0
Name: 0, dtype: float64
提供方1类型         0.0
提供方2类型         0.0
回应偏好1          0.1
回应偏好2          0.1
random_seed    2.0
Name: 100, dtype: float64
提供方1类型         0.0
提供方2类型         0.0
回应偏好1          0.1
回应偏好2          0.1
random_seed    3.0
Name: 200, dtype: float64
提供方1类型         0.0
提供方2类型         0.0
回应偏好1          0.1
回应偏好2          0.1
random_seed    4.0
Name: 300, dtype: float64
提供方1类型         0.0
提供方2类型         0.0
回应偏好1          0.1
回应偏好2          0.1
random_seed    5.0
Name: 400, dtype: float64
提供方1类型         0.0
提供方2类型         0.0
回应偏好1          0.1
回应偏好2          0.1
random_seed    6.0
Name: 500, dtype: float64
提供方1类型         0.0
提供方2类型         0.0
回应偏好1          0.1
回应偏好2          0.1
random_seed    7.0
Name: 600, dtype: float64
提供方1类型         0.0
提供方2类型         0.0
回应偏好1          0.1
回应偏好2          0.1
random_seed    8.0
Name: 700, dtype: float64
提供方1类型         0.0
提供方2类型         

提供方1类型          0.0
提供方2类型          0.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    76.0
Name: 7500, dtype: float64
提供方1类型          0.0
提供方2类型          0.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    77.0
Name: 7600, dtype: float64
提供方1类型          0.0
提供方2类型          0.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    78.0
Name: 7700, dtype: float64
提供方1类型          0.0
提供方2类型          0.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    79.0
Name: 7800, dtype: float64
提供方1类型          0.0
提供方2类型          0.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    80.0
Name: 7900, dtype: float64
提供方1类型          0.0
提供方2类型          0.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    81.0
Name: 8000, dtype: float64
提供方1类型          0.0
提供方2类型          0.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    82.0
Name: 8100, dtype: float64
提供方1类型          0.0
提供方2类型          0.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    83.0
Name: 8200,

提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    203.0
Name: 20200, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    204.0
Name: 20300, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    205.0
Name: 20400, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    206.0
Name: 20500, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    207.0
Name: 20600, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    208.0
Name: 20700, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    209.0
Name: 20800, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2 

提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    294.0
Name: 29300, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    295.0
Name: 29400, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    296.0
Name: 29500, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    297.0
Name: 29600, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    298.0
Name: 29700, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    299.0
Name: 29800, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    300.0
Name: 29900, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2 

提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    366.0
Name: 36500, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    367.0
Name: 36600, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    368.0
Name: 36700, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    369.0
Name: 36800, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    370.0
Name: 36900, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    371.0
Name: 37000, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2            0.1
random_seed    372.0
Name: 37100, dtype: float64
提供方1类型           0.0
提供方2类型           0.0
回应偏好1            0.1
回应偏好2 

...

提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    21.0
Name: 902000, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    22.0
Name: 902100, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    23.0
Name: 902200, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    24.0
Name: 902300, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    25.0
Name: 902400, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    26.0
Name: 902500, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    27.0
Name: 902600, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.1
回应偏好2           0.1
random_seed    28

提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.5
回应偏好2           0.1
random_seed    28.0
Name: 911700, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.5
回应偏好2           0.1
random_seed    29.0
Name: 911800, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.5
回应偏好2           0.1
random_seed    30.0
Name: 911900, dtype: float64
提供方1类型         0.0
提供方2类型         1.0
回应偏好1          0.5
回应偏好2          0.5
random_seed    1.0
Name: 912000, dtype: float64
提供方1类型         0.0
提供方2类型         1.0
回应偏好1          0.5
回应偏好2          0.5
random_seed    2.0
Name: 912100, dtype: float64
提供方1类型         0.0
提供方2类型         1.0
回应偏好1          0.5
回应偏好2          0.5
random_seed    3.0
Name: 912200, dtype: float64
提供方1类型         0.0
提供方2类型         1.0
回应偏好1          0.5
回应偏好2          0.5
random_seed    4.0
Name: 912300, dtype: float64
提供方1类型         0.0
提供方2类型         1.0
回应偏好1          0.5
回应偏好2          0.5
random_seed    5.0
Name: 912400, dtype: f

提供方1类型         0.0
提供方2类型         1.0
回应偏好1          0.9
回应偏好2          0.1
random_seed    7.0
Name: 918600, dtype: float64
提供方1类型         0.0
提供方2类型         1.0
回应偏好1          0.9
回应偏好2          0.1
random_seed    8.0
Name: 918700, dtype: float64
提供方1类型         0.0
提供方2类型         1.0
回应偏好1          0.9
回应偏好2          0.1
random_seed    9.0
Name: 918800, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.1
random_seed    10.0
Name: 918900, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.1
random_seed    11.0
Name: 919000, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.1
random_seed    12.0
Name: 919100, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.1
random_seed    13.0
Name: 919200, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.1
random_seed    14.0
Name: 919300

提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.9
random_seed    23.0
Name: 926200, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.9
random_seed    24.0
Name: 926300, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.9
random_seed    25.0
Name: 926400, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.9
random_seed    26.0
Name: 926500, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.9
random_seed    27.0
Name: 926600, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.9
random_seed    28.0
Name: 926700, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.9
random_seed    29.0
Name: 926800, dtype: float64
提供方1类型          0.0
提供方2类型          1.0
回应偏好1           0.9
回应偏好2           0.9
random_seed    30


KeyboardInterrupt



In [ ]:
1

In [203]:
#data.to_csv(r'C:\Users\付应超\Desktop\小论文数据-在线评论\数据-jais修改\数据\data_all_process.csv', index=False, encoding='utf-8-sig')
#开始
data=pd.read_csv(r'C:\Users\付应超\Desktop\小论文数据-在线评论\数据-jais修改\数据\data_all_process.csv')
data.columns

Index(['step', '提供方1类型', '提供方2类型', '提供方1利他程度', '提供方2利他程度', 'OP1', 'OP2', 'PCIP1', 'PCIP2', 'ARPP1', 'ARPP2', 'ARSP1', 'ARSP2', '服务人数1', '服务人数2', '总服务人数', '评论1', '评论2', '社会总福利', '平台收益', '提供方收益1', '提供方收益2', '提供方总收益', '消费者收益1', '消费者收益2', '消费者总收益', '实际到达人数', '办理入住数1', '办理入住数2', '办理入住总数', '被激励好评数量1', '被激励好评数量2', '获得激励数量1', '获得激励数量2', '好评占比1', '好评占比2', '总好评数量占比', 'PERP1', 'PERP2', 'market_type', '评论1_lag1', '评论2_lag1', '评论1_当期', '评论2_当期', 'RP1', 'RP2', '累计评论数量1', '累计评论数量2', '当期评论数量1', '当期评论数量2', '好评无回应占比1', '差评无回应占比1', '价值共创占比1', '评论端价值下降占比1', '回应端价值下降占比1', '价值共毁占比1', '好评无回应占比2', '差评无回应占比2', '价值共创占比2', '评论端价值下降占比2', '回应端价值下降占比2', '价值共毁占比2', '好评无回应1', '差评无回应1', '价值共创1', '评论端价值下降1', '回应端价值下降1', '价值共毁1', '好评无回应2', '差评无回应2', '价值共创2', '评论端价值下降2', '回应端价值下降2', '价值共毁2', '累计好评1'], dtype='object')

In [204]:
data[['好评无回应1', '差评无回应1', '价值共创1', '评论端价值下降1', '回应端价值下降1', '价值共毁1']].describe()

,好评无回应1,差评无回应1,价值共创1,评论端价值下降1,回应端价值下降1,价值共毁1
count,99000.000000,99000.000000,99000.000000,99000.000000,99000.000000,99000.000000
mean,3238.904535,549.732535,751.834242,167.199000,48.815121,167.151414
std,1900.482331,323.683560,441.360977,99.297016,29.424148,99.016556
min,28.000000,0.000000,2.000000,0.000000,0.000000,0.000000
25%,1582.000000,289.000000,367.000000,87.000000,23.000000,87.000000
50%,3247.000000,554.000000,753.000000,168.000000,48.000000,168.000000
75%,4841.000000,788.000000,1123.000000,240.000000,73.000000,239.000000
max,8552.000000,1675.000000,1981.000000,549.000000,154.000000,524.000000


In [213]:
'''# 计算均值和标准差
columns=['好评无回应占比1', '差评无回应占比1', '价值共创占比1', '评论端价值下降占比1', '回应端价值下降占比1', '价值共毁占比1','RP1']
means = data[columns].mean()
stds = data[columns].std()

# 定义极端值的阈值
cutoff = stds * 3

# 计算每列的最小和最大有效值
lower_bounds = means - cutoff
upper_bounds = means + cutoff

# 使用条件筛选移除极端值
for column in columns:
    data = data[(data[column] > lower_bounds[column]) & (data[column] < upper_bounds[column])]'''

In [ ]:
#变量间关系可视化
plt.rcParams['font.sans-serif'] = ['SimHei']
fig, ax = plt.subplots(figsize=(10, 6))
dt=data
#gb=dt.groupby(['PERP1','PERP2'])
#dt=gb.get_group((0.9, 0.9))
x='累计好评数量'
y='办理入住数1'
sns.scatterplot(x=x, y=y, data=dt, ax=ax)
#sns.scatterplot(x=x, y='办理入住数1_per', data=dt, ax=ax)
sns.regplot(x=x, y=y, data=dt, order=1, scatter=False, color='red', ax=ax)
# 手动设置y轴的显示范围
#ax.set_ylim(0.1, 0.2)  # 根据你的数据和需要调整这些值
plt.xlabel(x)
plt.ylabel(y)
plt.title('Scatter Plot of Step vs Value with Non-linear Fit')
plt.show()

In [275]:
#由于评论占比之间存在严重共线性问题，使用pls回归
#使用交叉验证确定最大化模型性能的成分数
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

data['办理入住数1_per']=data['办理入住数1']/data['实际到达人数']
data['累计差评1']=data['累计评论数量1']-data['累计好评1']
data['提升型回应数量1']=data['价值共创1']+data['评论端价值下降1']
data['破坏型回应数量1']=data['价值共毁1']+data['回应端价值下降1']
data['累计好评2']=data['好评无回应2']+data['价值共创2']+data['回应端价值下降2']
data['累计差评2']=data['累计评论数量2']-data['累计好评2']
data['提升型回应数量2']=data['价值共创2']+data['评论端价值下降2']
data['破坏型回应数量2']=data['价值共毁2']+data['回应端价值下降2']
data['Usp1']=0.3+(1+data['OP1'])*data['ARSP1']-data['ARPP1']
data['Usp2']=0.3+(1+data['OP2'])*data['ARSP2']-data['ARPP2']
# 提取自变量和因变量
y = data['办理入住数1_per'].values
X = data[['累计好评1', '累计差评1', '提升型回应数量1', '破坏型回应数量1','Usp1', 'Usp2']].values

# 标准化自变量和因变量
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1))

# 定义最大成分数
max_components = min(X.shape)

# 记录交叉验证得分
cv_scores = []

# 测试不同的成分数
for n in range(1, max_components+1):
    pls = PLSRegression(n_components=n)
    scores = cross_val_score(pls, X, y, cv=10, scoring='neg_mean_squared_error')#neg_mean_squared_error、r2、explained_variance
    cv_scores.append(scores.mean())

# 选择最优成分数
optimal_components = np.argmax(cv_scores) + 1
print(f"最优成分数: {optimal_components}")

# 可视化交叉验证得分
import matplotlib.pyplot as plt

plt.plot(range(1, max_components+1), cv_scores, marker='o')
plt.xlabel('成分数')
plt.ylabel('交叉验证得分（负均方误差）')
plt.title('选择最优成分数')
plt.grid(True)
plt.show()


最优成分数: 2


In [257]:
#基于方差解释率
data['办理入住数1_per']=data['办理入住数1']/data['实际到达人数']
data['累计差评1']=data['累计评论数量1']-data['累计好评1']
data['提升型回应数量1']=data['价值共创1']+data['评论端价值下降1']
data['破坏型回应数量1']=data['价值共毁1']+data['回应端价值下降1']
data['累计好评2']=data['好评无回应2']+data['价值共创2']+data['回应端价值下降2']
data['累计差评2']=data['累计评论数量2']-data['累计好评2']
data['提升型回应数量2']=data['价值共创2']+data['评论端价值下降2']
data['破坏型回应数量2']=data['价值共毁2']+data['回应端价值下降2']
data['Usp1']=0.3+(1+data['OP1'])*data['ARSP1']-data['ARPP1']
data['Usp2']=0.3+(1+data['OP2'])*data['ARSP2']-data['ARPP2']
# 提取自变量和因变量
y = data['办理入住数1_per'].values
X = data[['累计好评1', '累计差评1', '提升型回应数量1', '破坏型回应数量1','Usp1', 'Usp2']].values

# 标准化自变量和因变量
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1))

# 定义最大成分数
max_components = min(X.shape)

explained_variance = []

for n_components in range(1, max_components + 1):
    pls = PLSRegression(n_components=n_components)
    pls.fit(X, y)
    variance = np.sum(pls.x_scores_ ** 2) / np.sum(X ** 2)
    explained_variance.append(variance)

# 找到解释方差达到95%的最小成分数
threshold = 0.9
# 检查是否有满足条件的成分数
indices = np.where(np.array(explained_variance) >= threshold)[0]

# 如果没有找到，设置为None或其他合适的值
if indices.size > 0:
    optimal_variance_components = indices[0] + 1
else:
    optimal_variance_components = None

# 输出结果
if optimal_variance_components is not None:
    print(f"最优成分数 (解释方差率): {optimal_variance_components}")
else:
    print("没有找到达到指定阈值的成分数")

# 可视化解释方差率
plt.plot(range(1, max_components + 1), explained_variance, marker='o')
plt.axhline(y=threshold, color='r', linestyle='--', label='95%阈值')
plt.xlabel('成分数')
plt.ylabel('累积解释方差率')
plt.title('解释方差率 成分数选择')
plt.grid(True)
plt.legend()
plt.show()


没有找到达到指定阈值的成分数


In [286]:
#AIC/BIC信息准则
from sklearn.metrics import mean_squared_error
def calculate_aic_bic(y, y_pred, n_params):
    resid = y - y_pred
    sse = np.sum(resid ** 2)
    n = len(y)
    aic = n * np.log(sse/n) + 2 * n_params
    bic = n * np.log(sse/n) + np.log(n) * n_params
    return aic, bic

data['办理入住数1_per']=data['办理入住数1']/data['实际到达人数']
data['累计差评1']=data['累计评论数量1']-data['累计好评1']
data['提升型回应数量1']=data['价值共创1']+data['评论端价值下降1']
data['破坏型回应数量1']=data['价值共毁1']+data['回应端价值下降1']
data['累计好评2']=data['好评无回应2']+data['价值共创2']+data['回应端价值下降2']
data['累计差评2']=data['累计评论数量2']-data['累计好评2']
data['提升型回应数量2']=data['价值共创2']+data['评论端价值下降2']
data['破坏型回应数量2']=data['价值共毁2']+data['回应端价值下降2']
data['Usp1']=0.3+(1+data['OP1'])*data['ARSP1']-data['ARPP1']
data['Usp2']=0.3+(1+data['OP2'])*data['ARSP2']-data['ARPP2']
# 提取自变量和因变量
y = data['办理入住数1_per'].values
X = data[['累计好评1', '累计差评1', '提升型回应数量1', '破坏型回应数量1','ARPP1', 'ARPP2', 'ARSP1', 'ARSP2']].values

# 标准化自变量和因变量
scaler_X = StandardScaler()
scaler_y = StandardScaler()

#X = scaler_X.fit_transform(X)
#y = scaler_y.fit_transform(y.reshape(-1, 1))

# 定义最大成分数
max_components = min(X.shape)

aic_values = []
bic_values = []

for n_components in range(1, max_components + 1):
    pls = PLSRegression(n_components=n_components)
    pls.fit(X, y)
    y_pred = pls.predict(X)
    aic, bic = calculate_aic_bic(y, y_pred, n_components)
    aic_values.append(aic)
    bic_values.append(bic)

optimal_aic_components = np.argmin(aic_values) + 1
optimal_bic_components = np.argmin(bic_values) + 1

print(f"最优成分数 (AIC): {optimal_aic_components}")
print(f"最优成分数 (BIC): {optimal_bic_components}")
# 绘制AIC/BIC曲线
plt.plot(range(1, max_components + 1), aic_values, marker='o', label='AIC')
plt.plot(range(1, max_components + 1), bic_values, marker='s', label='BIC')
plt.xlabel('成分数')
plt.ylabel('信息准则值')
plt.title('AIC/BIC 成分数选择')
plt.legend()
plt.grid(True)
plt.show()


最优成分数 (AIC): 5
最优成分数 (BIC): 5


In [285]:
#使用bootstrap法估计pls回归系数的显著性
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import resample
# 提取自变量和因变量
y = data['提供方收益1'].values
X = data[['累计好评1', '累计差评1', '提升型回应数量1', '破坏型回应数量1','ARPP1', 'ARPP2', 'ARSP1', 'ARSP2']].values

# 标准化自变量和因变量
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1))

# 创建PLS回归模型
pls = PLSRegression(n_components=5)
pls.fit(X, y)

# 原始回归系数
original_coefficients = pls.coef_.flatten()

# 自助法参数
n_iterations = 1000  # 迭代次数
n_size = len(data)   # 数据集大小
bootstrapped_coefficients = []

# 自助法迭代
for i in range(n_iterations):
    # 重采样数据
    X_resampled, y_resampled = resample(X, y, n_samples=n_size, random_state=i)
    
    # 拟合PLS模型
    pls.fit(X_resampled, y_resampled)
    
    # 保存回归系数
    bootstrapped_coefficients.append(pls.coef_.flatten())

# 转换为numpy数组
bootstrapped_coefficients = np.array(bootstrapped_coefficients)

# 计算每个回归系数的置信区间
confidence_intervals = np.percentile(bootstrapped_coefficients, [2.5, 97.5], axis=0)

# 打印结果
print("原始回归系数:", original_coefficients)
print("95% 置信区间:")
for i, (ci_lower, ci_upper) in enumerate(zip(confidence_intervals[0], confidence_intervals[1])):
    print(f"  变量 {i+1}: [{ci_lower:.4f}, {ci_upper:.4f}]")

原始回归系数: [ 0.02621199 -0.02651069  0.01637415 -0.01660301 -0.99315705  0.53122007
  0.71887352 -0.46802211]
95% 置信区间:
  变量 1: [0.0175, 0.0345]
  变量 2: [-0.0357, -0.0171]
  变量 3: [0.0102, 0.0226]
  变量 4: [-0.0240, -0.0096]
  变量 5: [-1.0016, -0.9846]
  变量 6: [0.5234, 0.5392]
  变量 7: [0.7102, 0.7279]
  变量 8: [-0.4765, -0.4601]


In [250]:
data['办理入住数1_per']=data['办理入住数1']/data['实际到达人数']
data['累计差评1']=data['累计评论数量1']-data['累计好评1']
data['提升型回应数量1']=data['价值共创1']+data['评论端价值下降1']
data['破坏型回应数量1']=data['价值共毁1']+data['回应端价值下降1']
data['累计好评2']=data['好评无回应2']+data['价值共创2']+data['回应端价值下降2']
data['累计差评2']=data['累计评论数量2']-data['累计好评2']
data['提升型回应数量2']=data['价值共创2']+data['评论端价值下降2']
data['破坏型回应数量2']=data['价值共毁2']+data['回应端价值下降2']
data['Usp1']=0.3+(1+data['OP1'])*data['ARSP1']-data['ARPP1']
data['Usp2']=0.3+(1+data['OP2'])*data['ARSP2']-data['ARPP2']
formula='办理入住数1_per~累计好评1+累计差评1+提升型回应数量1+破坏型回应数量1+Usp1+Usp2'
model=ols(formula,data=data).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             办理入住数1_per   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                 1.738e+04
Date:                Thu, 08 Aug 2024   Prob (F-statistic):               0.00
Time:                        19:26:13   Log-Likelihood:                 45393.
No. Observations:               99000   AIC:                        -9.077e+04
Df Residuals:                   98993   BIC:                        -9.070e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4169      0.001    423.454      0.000       0.415       0.419
累计好评1       6.803e-06    3.4e-06      1.999      0.046    1.34e-07    1.35e-05
累计差评1      -1.413e-05   8.36e-06     -1.691      0.091   -3.05e-05    2.25e-06
提升型回应数量1   -1.385e-05   1.77e-05     -0.783      0.433   -4.85e-05    2.08e-05
破坏型回应数量1   -1.169e-05   3.69e-05     -0.317      0.752   -8.41e-05    6.07e-05
Usp1           1.2437      0.004    301.327      0.000       1.236       1.252
Usp2          -0.4610      0.004   -119.307      0.000      -0.469      -0.453
==============================================================================
Omnibus:                      708.851   Durbin-Watson:                   2.059
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              951.410
Skew:                          -0.106   Prob(JB):                    2.53e-207
Kurtosis:                       3.431   Cond. No.                     4.17e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.17e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [219]:
#共线性检验
calculate_vif(data,['累计好评1', '累计差评1', '提升型回应数量1', '破坏型回应数量1'])

,feature,VIF
0,累计好评1,1073.725689
1,累计差评1,309.983215
2,提升型回应数量1,1488.474192
3,破坏型回应数量1,359.452874


In [118]:
data.columns

Index(['step', '提供方1类型', '提供方2类型', '提供方1利他程度', '提供方2利他程度', 'OP1', 'OP2', 'PCIP1', 'PCIP2', 'ARPP1', 'ARPP2', 'ARSP1', 'ARSP2', '服务人数1', '服务人数2', '总服务人数', '评论1', '评论2', '社会总福利', '平台收益', '提供方收益1', '提供方收益2', '提供方总收益', '消费者收益1', '消费者收益2', '消费者总收益', '实际到达人数', '办理入住数1', '办理入住数2', '办理入住总数', '被激励好评数量1', '被激励好评数量2', '获得激励数量1', '获得激励数量2', '好评占比1', '好评占比2', '总好评数量占比', 'PERP1', 'PERP2', 'market_type', '评论1_lag1', '评论2_lag1', 'RP1', 'RP2', '累计评论数量1', '累计评论数量2', '好评无回应占比1', '差评无回应占比1', '价值共创占比1', '评论端价值下降占比1', '回应端价值下降占比1', '价值共毁占比1', '好评无回应占比2', '差评无回应占比2', '价值共创占比2', '评论端价值下降占比2', '回应端价值下降占比2', '价值共毁占比2', '对好评回应率1', '对差评回应率1', '对好评提升型回应率1', '对差评提升型回应率1', '累计好评占比1', '好评无回应1', '差评无回应1', '价值共创1', '评论端价值下降1', '回应端价值下降1', '价值共毁1', '好评无回应2', '差评无回应2', '价值共创2', '评论端价值下降2', '回应端价值下降2', '价值共毁2', '累计好评1', '办理入住数1_per', 'Usp1', 'Usp2', '累计差评1', '提升型回应数量1', '破坏型回应数量1', '累计好评2', '累计差评2', '提升型回应数量2', '破坏型回应数量2'], dtype='object')